In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie
import json
import pickle

In [2]:
%load_ext autoreload
%autoreload 1
%aimport Regions1, physio_def_2, numeric, Automatic1

from physio_def_2 import *
from Regions1 import Regions
from numeric import *
from Automatic1 import *

if "srdjan" not in os.getcwd():
    os.chdir(os.path.expanduser("~"))

outputStyle = {
    "color":"navy",
    "font-family":"Courier New",
    "font-size":"80%",
    }
infoStyle = {
    "font-size":"80%",
    "color":"grey",
    }
bodyStyle = {
    }

baseFig = getFigure(w=200,h=150)

## global vars
test = True#"srdjan" in os.getcwd()
debug = False#"srdjan" in os.getcwd()

if test:
    %config InlineBackend.figure_format = 'retina'

from plotly.colors import DEFAULT_PLOTLY_COLORS

In [3]:
def tryexcept(x,default):
    try:
        return eval(x)
    except:
        return default

In [4]:
timescales = [.3,.4,.5,.7]+sorted([i*j for i in [1,2,3,4,5,7] for j in [1,10]])[:-3]+ sorted([i*j*60 for i in [1,1.5,2,3,5] for j in [1]])
timescales = np.array(timescales)

In [5]:
markerDict = { .3: "ultrafast",2:"fast",60:"slow"}

In [6]:
folder = "/data/prototype/"

allRecs = sorted([os.path.join(folder,el) for el in os.listdir(folder) if el.endswith("lif")])

from dash import no_update

exampleLif = "/data/prototype/Experiment52c.lif"

In [7]:
app = JupyterDash(__name__,
                  width=1000,
                  height=1000,
                 )

filterSlider = dcc.Slider(
                    id = "filter-slider",
                    min=0,
                    max=len(timescales)-1,
                    value=int(np.where(timescales>=3)[0][0]),
                    marks = {tryexcept(f"int(np.where(timescales>={ts})[0][0])",len(timescales)-1):{"label":markerDict[ts]} for ts in markerDict},
#                     marks={int(i_ultrafast): {"label":"ultrafast"},
#                            int(i_fast)     : {"label":"fast"},
#                            i_slow     : {"label":"slow"},
#                            (len(timescales)-1):{"label":"raw"}
#                           },
                    updatemode='drag',
                    )
SelectedRoisHidden = html.Div([
           "Selected ROIs:",
            dcc.Input(id="selected-rois",
                type="text",
                debounce=False,
                size=6,
                value="",
             ), 
            html.Button("Calc all Peaks",id="calc-peaks"),
            html.Div("",id="calc-peaks-feedback"),
        ],style={"display": "inline-box" if debug else "hidden",**infoStyle})
# CalcPeaksButtonHidden = html.Div(html.Button("Calc all Peaks",id="calc-peaks"),style={"display": "inline-box" if debug else "hidden"})

rangePickers = html.Div([dcc.RangeSlider(
                    id = "range-slider-%i"%i,
                    min=0.,
                    max=1.,
                    value=[0.02,.98] if i==0 else [0,0],
                    updatemode='mouseup',
                    included=True,
                    step=.001,
                    ) for i in range(3)],
                    style={"width":560, "padding-left":55})

APP_LAYOUT = [
    
    html.H1(children='CTN Analytics',style=bodyStyle),
    html.Div("how do you like the name? :-)",style=infoStyle),
    html.Br(),
    ####################################
    html.Div('Select the experiment',style={**bodyStyle, "display":"inline-box"}),
    dcc.Dropdown(options = [{"value":path,"label":path} for path in allRecs], id="filepath-dropdown", value = exampleLif if (test and exampleLif in allRecs) else None, style={**bodyStyle,"width":"70%"}),
    html.Div('Select the series',style={**bodyStyle, "display":"inline-box"}),
    dcc.Dropdown(options = [{"value":None,"label":None}], id="series-dropdown", style={**bodyStyle,"width":"70%"}),
    html.Div([
        html.Div([
            'Select filer size',
            dcc.RadioItems(options = [{"value":None,"label":None}], id="pickle-radio", labelStyle={"display":"block"})
        ],style={**bodyStyle, "display":"inline-block", "vertical-align":"text-top"}),
        html.Div(id="pickle-previews", children = ["None"],style={"display":"inline-block","padding-left":"20px","vertical-align":"text-top",}),
    ],style={"padding-top":"20px"}),
    html.Div(id="pickle-feedback",style={**outputStyle}),
       
]
APP_LAYOUT += [html.Div([
        html.Div([
            k if debug else None,
            dcc.Graph(id=k, figure=getFigure(300,400)),
            {"spike-durations":filterSlider,"roi-selector":SelectedRoisHidden, "range-pickers":rangePickers}.get(k)
                 ],style={"max-width":"550px","max-height":"550px","border":"thin grey solid",}
                ) for k in ks
    ],
        style={"display":"flex", "flex-wrap":"wrap","align-items":"top","flex-shrink":3}
    ) for ks in [["roi-selector","range-pickers"],["spike-durations","raster"],["roi-hover"]*int(debug)]]

In [8]:
# callbacks
@app.callback(
    [Output("series-dropdown","value"),
    Output("series-dropdown","options")],
    [Input("filepath-dropdown","value")]
)
def serve_series(pathToRec):
    if pathToRec is None:
        return no_update
    if len(pathToRec)==0:
        return no_update
    analysisFolder = pathToRec+"_analysis"
    opts = [{"value":os.path.join(analysisFolder,el), "label":el} for el in sorted(os.listdir(analysisFolder)) if os.path.isdir(os.path.join(analysisFolder,el))]
    global pathToSer
    pathToSer = (opts[0]["value"] if len(opts) and test else None)
    return pathToSer, opts

@app.callback(
    [Output("pickle-radio","value"),
     Output("pickle-radio","options"),
     Output("pickle-previews","children")
    ],
    [Input("series-dropdown","value")]
)
def serve_pickles(pathToSer,width=200, height=200):
    import base64
    from collections import OrderedDict 
    if pathToSer is None:
        return no_update
    if len(pathToSer)==0:
        return no_update
    preview = OrderedDict()
    options = []
    for f in sorted(os.listdir(pathToSer))[::-1]:
        if f.endswith("pkl"):
            k = f.split("_")[0].replace(".","+")
            pathToRoi = os.path.join(pathToSer,f)
            options += [{ "label": k, "value": pathToRoi }]
            previewFile = pathToRoi.replace(f,".image_"+f.replace("_rois.pkl",".png"))
            if os.path.isfile(previewFile):
                encoded_image = base64.b64encode(open(previewFile, 'rb').read())
                preview[k] = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width="%ipx"%(width*.9),height="%ipx"%(height*.8+20))

    if len(options) and test:
        val = options[0]["value"]
    else:
        val = None
    previews = [html.Div(children=[html.Div(k,style={"padding-left":"20px"}), preview[k]], style=dict(
        height="%ipx"%height,
        width="%ipx"%width,
        display="inline-block",
        border='thin lightgrey solid'
    )) for k in preview]
    return val, options, html.Div(previews,style={"width":"%ipx"%(width*2.3)})

@app.callback(
    [Output("pickle-feedback","children"),
     Output("roi-selector","figure"),
     Output("calc-peaks","n_clicks"),
    ],
    [Input("pickle-radio","value")]
)
def import_pickle(path):
    if path is None:
        return no_update
    if len(path)==0:
        return no_update
    global regions
    with open(path,"rb") as f:
        regions = pickle.load(f)
    regions.update()
    feedback = [
        "Regions imported successfully.",
        "Original movie:",
        html.Div(children=[
            "dimension (T,X,Y): (%i, %i, %i)"%((len(regions.time), )+regions.image.shape),
            html.Br(),
            "duration (h:m:s): "+str(pd.Timedelta(regions.time.max(), unit="s")).split()[-1].split(".")[0],
            html.Br(),
            "frequency (Hz): %.3g"%regions.Freq,
        ],style={'padding-left': '30px'}),
        "Number of ROIs: %i"%len(regions.df),
    ]
    feedback = sum([[el, html.Br()] for el in feedback],[])
    roisImage = showRoisOnly(regions,indices=regions.df.index, im=regions.statImages[regions.mode])
    return feedback,roisImage,1

In [9]:
@app.callback(
    Output("spike-durations", "figure"),
    [Input("filter-slider", "value")]+[Input(rp.id, "value") for rp in rangePickers.children]
)
def plotPeakStats_callback(i, *ranges):
    timeScale = timescales[i]
    global regions
    k = "%g"%timeScale
    try:
        regions.peaks[k]
    except:
        regions.calc_peaks(timeScale)
    t0,te = regions.time[[0,-1]]
    ranges = [sorted(r) for r in ranges]
#     ranges = [r for r in ranges if r[1]!=r[0]]
    ranges = np.array(ranges)*(te-t0)+t0
    trfig = regions.show_scatter_peaks(timeScale, ranges)
    return trfig

In [10]:
@app.callback(
    Output("calc-peaks-feedback", "children"),
    [Input("filter-slider", "value")]+[Input(rp.id, "value") for rp in rangePickers.children]
)
def tmp_callback(i, *ranges):
    timeScale = timescales[i]
    return str(i)+" "+str(ranges)

In [11]:
@app.callback(
    Output("selected-rois", "value"),
    [Input("roi-selector", "selectedData")],
    )
def showSelected(selData):
    if selData is None:
        return "all"
    return ",".join(p["hovertext"] for p in selData["points"])

In [12]:
@app.callback(
    Output("range-pickers", "figure"),
    [Input("selected-rois", "value")]+[Input(rp.id, "value") for rp in rangePickers.children],
    )
def showTrace(selData,*ranges,showFreq = 2):
    global regions
    if selData is None:
        return baseFig
    if selData=="all":
        indices = regions.df.index
    else:
        indices = np.array(selData.split(",")).astype(int)
    nRebin = int(regions.Freq/showFreq)
    t = regions.time
    y = np.sum([regions.df.loc[i,"trace"]*regions.df.loc[i,"size"] for i in indices],axis=0)/regions.df["size"].sum()
    if nRebin>1:
        y = rebin(y, nRebin)
        t = rebin(t, nRebin)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t,y=y, line_width=.5, opacity=1, line_color="darkred"))
    ranges = [sorted(r) for r in ranges]
    ranges = [np.array(r)*(t[-1]-t[0])+t[0] if r[1]-r[0]>.01 else [0,1] for r in ranges ]
    fig.update_layout(
        shapes = [
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=r[0],
                y0=0,
                x1=r[1],
                y1=1,
                fillcolor=DEFAULT_PLOTLY_COLORS[i],
                opacity=0.3,
                layer="below",
                line_width=0,
            ) for i,r in enumerate(ranges)])
    fig.update_layout({
        "plot_bgcolor":"white",
        "width":600,
        "height":400,
        "xaxis":dict(
            mirror=True,
            ticks='outside',
            showline=True,),
#         "dragmode":'drawrect',
        "margin": dict(t=50,r=10)
    })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="outside", ticklen=3, nticks=10, title="time [s]", gridcolor="lightgrey")
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="outside", ticklen=3, gridcolor="lightgrey")
#     dragmode=,
#     newshape_line_color='cyan',
#     title_text='Draw a path to separate versicolor and virginica'
    return fig#.show(config={'modeBarButtonsToAdd':['drawline',
#                                     'drawopenpath',
#                                     'drawclosedpath',
#                                     'drawcircle',
#                                     'drawrect',
#           `                          'eraseshape'
#                                    ]})

In [13]:
@app.callback(
    Output("raster", "figure"),
    [Input("filter-slider", "value")]
)
def plotRaster_callback(i, npoints = 1000):
    timeScale = timescales[i]
    global regions
    k = "%g"%timeScale
    try:
        regions.peaks[k]
    except:
        regions.calc_peaks(timeScale)
    df = regions.peaks[k]
    C = regions.df
    rr = np.zeros((len(C),npoints))
    tt = pd.Series(np.linspace(0,regions.time.max(),npoints))
    for i,ix in enumerate(C.index):
        ddf = df.query(f"roi=={ix}")
        for _,row in ddf.iterrows():
            rr[i,tt.between(row.t0,row.t0+row.iloc[1])] = 1
    
    fig = go.Figure(go.Heatmap(
            x=tt,
            z=rr,
            showscale=False,
            hoverinfo="text",text=[[str(i)] for i in regions.df.index]
        ))
    fig.update_yaxes(title_text='roi id')
    fig.update_xaxes(title_text='time [s]')
    fig.update_layout({
        "width":  600,
        "height": 400,
        "margin": dict(l=20, r=10, t=50, b=20),
    })
        # fig.update_xaxes(showticklabels=False)
    return fig

In [14]:
# @app.callback(
#     Output("calc-peaks-feedback","children"),
#     [Input("calc-peaks", "n_clicks")])
# def calcPeaks_callback(i):
#     if i>0:
#         global regions
#         for ts in timescales[::-1]:
#             try:
#                 regions.peaks[k]
#             except:
#                 regions.calc_peaks(ts)
#         return "Done"
#     else:
#         return no_update

In [15]:
# app.layout = html.Div(children=APP_LAYOUT,
#                       style={"family":"Arial"}
#                      )
# app

In [16]:
app.layout = html.Div(children=APP_LAYOUT)
app._repr_html_() 
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')

In [18]:
regions.peaks.keys()

dict_keys(['4', '5', '3', '2', '1'])